<a href="https://colab.research.google.com/github/mjbernalv/trabajo2-si7006-252-perfilamiento/blob/main/pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
from pyspark.sql.functions import col
from google.colab import drive
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, DoubleType
import shutil
import glob
from pyspark.sql import functions as F, Window
import re

def limpiar_nombre(nombre):
    if nombre is None:
        return "Desconocido"
    # Reemplazar cualquier carácter que no sea letra o número por "_"
    return re.sub(r'[^A-Za-z0-9_]', '_', nombre.strip())

In [24]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
spark = SparkSession.builder \
    .appName("EDA_Dataset_PI") \
    .getOrCreate()

print("Spark iniciado:", spark)

Spark iniciado: <pyspark.sql.session.SparkSession object at 0x7e982c5bfbc0>


In [26]:
ruta_base = "/content/drive/MyDrive/Dataset PI/"

detalle_poliza = ruta_base + "Raw/detalle_poliza.csv"
detalle_producto = ruta_base + "Raw/detalle_producto.csv"
detalle_cliente = ruta_base + "Raw/detalle_cliente.csv"

# Leer con inferencia de esquema y header
df_poliza = spark.read.option("header", "true").option("inferSchema", "true").csv(detalle_poliza)
df_producto = spark.read.option("header", "true").option("inferSchema", "true").csv(detalle_producto)
df_clientes = spark.read.option("header", "true").option("inferSchema", "true").csv(detalle_cliente)

# Mostrar confirmación
print("Poliza:", df_poliza.count(), "filas,", len(df_poliza.columns), "columnas")
print("Producto:", df_producto.count(), "filas,", len(df_producto.columns), "columnas")
print("Clientes:", df_clientes.count(), "filas,", len(df_clientes.columns), "columnas")



Poliza: 226254 filas, 28 columnas
Producto: 562 filas, 11 columnas
Clientes: 253062 filas, 8 columnas


In [27]:
# Unión de poliza con clientes
df_poliza_clientes = df_poliza.join(
    df_clientes,
    on="codCliente",  # clave común
    how="left"
)

# Unión del resultado anterior con producto
df_final = df_poliza_clientes.join(
    df_producto,
    on="codProducto",  # clave común
    how="left"
)

# Mostrar resultado
print("Filas finales:", df_final.count())
print("Columnas finales:", len(df_final.columns))
df_final.show(5, truncate=False)

# Ruta donde guardaremos la tabla unida
ruta_salida = "/content/drive/MyDrive/Dataset PI/tabla_merge.csv"

# Guardar el dataframe como CSV con encabezado
df_final.coalesce(1) \
    .write.mode("overwrite") \
    .option("header", "true") \
    .csv(ruta_salida)

print(" Tabla final guardada en:", ruta_salida)



# Buscar el archivo generado dentro de la carpeta
archivo_generado = glob.glob(ruta_salida + "/part-*.csv")[0]

# Moverlo a la misma carpeta con el nombre tabla_final.csv
ruta_final = "/content/drive/MyDrive/Dataset PI/tabla_merge_unida.csv"
shutil.move(archivo_generado, ruta_final)

# Eliminar la carpeta vacía
shutil.rmtree(ruta_salida)


Filas finales: 226254
Columnas finales: 45
+-----------+----------+---------+-------------------+-----------+----------+-------------------+----------+-----------------+------+----------+-------------+---------------------+-------------------+----------------+-----------+------------------+----------+-----------+---------------+--------+----------+------------------------+--------------------+----------------+----------------------------+------------------+------------------+--------------------------+----------------+-------------------+----+------------------------------+------------+------------+------------------+-------+----------+-------------+-------------+--------------------------------+-------------------+----------+----------+------------------+
|codProducto|codCliente|codPoliza|estadoProcesamiento|fechaInicio|fechaFin  |origenProcesamiento|estadoDato|formaPagoVigencia|IVA   |valorTotal|IVA_Periodico|Valor_Total_Periodico|fechaInicioVigencia|fechaFinVigencia|estado     |Esta

In [28]:
columnas_a_eliminar = [
    "codProducto",
    "estadoProcesamiento",
    "origenProcesamiento",
    "estadoDato",
    "IVA",
    "IVA_Periodico",
    "fechaInicio",
    "fechaFin",
    "EstadoRenovaciones",
    "tipoPoliza",
    "Fecha",
    "valorTotalSinIva",
    "Valor_Total_Periodico_SinIva",
    "TasaIVA",
    "TasaIVA_Periodico",
    "fechaNacCliente",
    "FechaCumple",
    "nomTipoDocumento",
    "Tiene poliza",
    "codRamo",
    "seRenuevaRamo",
    "nomGrupo",
    "nomCompania",
    "GrupoRamo",
    "nomRamo",
    "NegociosEspeciales",
    "aseguradora",
    "tomador"
    "nomProducto"
]
# Eliminar columnas (solo si existen en el dataframe)
df_final = df_final.drop(*[c for c in columnas_a_eliminar if c in df_final.columns])

print("Columnas eliminadas correctamente")
print("Número de columnas restantes:", len(df_final.columns))


Columnas eliminadas correctamente
Número de columnas restantes: 19


In [29]:


df_final = df_final.withColumn(
    "Ramo",
    F.when(F.col("Ramo").isin(
        "Salud Familiar", "Enfermedades Graves", "Salud Colectivo",
        "Asistencia Medica", "Medicina Prepagada"
    ), "Salud")
    .when(F.col("Ramo").isin(
        "Vida Individual", "Vida Grupo", "Educacion", "Pension",
        "Accidentes Personales", "Juvenil", "Exequial"
    ), "Vida")
    .when(F.col("Ramo").isin(
        "Automoviles", "Soat"
    ), "Autos")
    .when(F.col("Ramo").isin(
        "Cumplimiento", "Arrendamiento"
    ), "Cumplimiento")
    .when(F.col("Ramo").isin(
        "Incendio", "Responsabilidad Civil", "Hogar", "Transporte Mercancia",
        "Transporte Valores", "Maquinaria Y Equipo", "Manejo", "Copropiedad",
        "Montaje", "Construcción", "Navegacion",
        "R.C. Parqueaderos Y Talleres", "Agricola"
    ), "Patrimoniales")
    .otherwise("Otros")
)

In [30]:
# Reemplazamos espacios y caracteres raros por guiones bajos
for old_name in df_final.columns:
    new_name = old_name.strip().replace(" ", "_").replace("-", "_")
    df_final = df_final.withColumnRenamed(old_name, new_name)


In [31]:
df_final = df_final.withColumn("Edad", col("Edad").cast(IntegerType()))
df_final = df_final.withColumn("valorTotal", col("valorTotal").cast(DoubleType()))


In [32]:
df_final.show(5, truncate=False)

+----------+---------+-----------------+----------+---------------------+-------------------+----------------+-----------+---------------+--------+------------------------+--------------------+-------------------+----+------------+------------+------------------+-------------------+-----+
|codCliente|codPoliza|formaPagoVigencia|valorTotal|Valor_Total_Periodico|fechaInicioVigencia|fechaFinVigencia|estado     |OpracionesGrupo|Vigencia|EstadoRenovacionesinicio|ciudad              |tipoVinculacion    |Edad|Tipo_Empresa|tomador     |nomProducto       |nomGrupoEmpresarial|Ramo |
+----------+---------+-----------------+----------+---------------------+-------------------+----------------+-----------+---------------+--------+------------------------+--------------------+-------------------+----+------------+------------+------------------+-------------------+-----+
|153029    |155142   |Anual            |-4043.0   |-4043.0              |2025-06-25         |2025-06-29      |Cancelada  |Cancelad

operaciones= nuevas and renovaciones
estado!= cancelado
valor >=0
vigencias != otros



crear una columna que coja cada policza y marque como 1 la mas vieja 2 la que sigue y asi

hacer un desc rank

In [33]:
df_final_filtered = df_final.filter(
    (col("OpracionesGrupo").isin("Nueva", "Renovación")) &
    (col("EstadoRenovaciones") != "Cancelada") &
    (col("valorTotal") > 0) &
    (col("Vigencia") != "Otros")
)

print("Número de filas después del filtrado:", df_final_filtered.count())
df_final_filtered.show(5, truncate=False)

Número de filas después del filtrado: 115745
+----------+---------+-----------------+-----------+---------------------+-------------------+----------------+-------+---------------+--------+------------------------+--------------------+-------------------+----+------------+------------+---------------------------+-------------------+-------------+
|codCliente|codPoliza|formaPagoVigencia|valorTotal |Valor_Total_Periodico|fechaInicioVigencia|fechaFinVigencia|estado |OpracionesGrupo|Vigencia|EstadoRenovacionesinicio|ciudad              |tipoVinculacion    |Edad|Tipo_Empresa|tomador     |nomProducto                |nomGrupoEmpresarial|Ramo         |
+----------+---------+-----------------+-----------+---------------------+-------------------+----------------+-------+---------------+--------+------------------------+--------------------+-------------------+----+------------+------------+---------------------------+-------------------+-------------+
|9853      |15214    |Mensual          |1.5

In [34]:
# Eliminar filas donde valorTotal sea 0 o nulo
df_final = df_final.filter((F.col("valorTotal") > 0) & (F.col("valorTotal").isNotNull()))

# Eliminar duplicados por las columnas clave
df_final = df_final.dropDuplicates(["codCliente", "codPoliza", "fechaInicioVigencia", "fechaFinVigencia", "Ramo"])

# (Opcional) Reordenar para verificar
df_final = df_final.orderBy("codCliente", "codPoliza", "fechaInicioVigencia")

print("Número de registros:", df_final.count())

Número de registros: 83574


In [35]:
# Asegurar que la columna de fecha sea tipo date
df_final = df_final.withColumn(
    "fechaInicioVigencia",
    F.to_date("fechaInicioVigencia", "dd/MM/yyyy")
)

# Crear ventana por cliente y póliza, ordenada por fecha de inicio
window_spec = Window.partitionBy("codCliente", "codPoliza").orderBy(F.asc("fechaInicioVigencia"))

# Agregar columna de orden (opcional, para validar el orden)
df_final = df_final.withColumn("orden_vigencia", F.row_number().over(window_spec))

# Mostrar resultados ordenados globalmente para ver el efecto
df_final = df_final.orderBy("codCliente", "codPoliza", "orden_vigencia")

#traer solo las primeras vigencias
df_final=df_final.filter(F.col("orden_vigencia") == 1)

df_final.show( truncate=False)


+----------+---------+-----------------+-----------+---------------------+-------------------+----------------+-------+---------------+--------+------------------------+--------+-------------------+----+------------+------------+--------------------------------------------------------+-------------------+-------------+--------------+
|codCliente|codPoliza|formaPagoVigencia|valorTotal |Valor_Total_Periodico|fechaInicioVigencia|fechaFinVigencia|estado |OpracionesGrupo|Vigencia|EstadoRenovacionesinicio|ciudad  |tipoVinculacion    |Edad|Tipo_Empresa|tomador     |nomProducto                                             |nomGrupoEmpresarial|Ramo         |orden_vigencia|
+----------+---------+-----------------+-----------+---------------------+-------------------+----------------+-------+---------------+--------+------------------------+--------+-------------------+----+------------+------------+--------------------------------------------------------+-------------------+-------------+--------

In [36]:
#Refined
# Ruta donde guardaremos la tabla unida
ruta_salida = "/content/drive/MyDrive/Dataset PI/Refined/Refined_temp.csv"

# Guardar el dataframe como CSV con encabezado
df_final.coalesce(1) \
    .write.mode("overwrite") \
    .option("header", "true") \
    .csv(ruta_salida)

print(" Tabla final guardada en:", ruta_salida)



# Buscar el archivo generado dentro de la carpeta
archivo_generado = glob.glob(ruta_salida + "/part-*.csv")[0]

# Moverlo a la misma carpeta con el nombre tabla_final.csv
ruta_final = "/content/drive/MyDrive/Dataset PI/Refined/Refined_data.csv"
shutil.move(archivo_generado, ruta_final)

# Eliminar la carpeta vacía
shutil.rmtree(ruta_salida)


 Tabla final guardada en: /content/drive/MyDrive/Dataset PI/Refined/Refined_temp.csv


In [37]:
# Crear ventana por cliente, ordenando por la fecha de inicio
window_spec = Window.partitionBy("codCliente").orderBy(F.asc("fechaInicioVigencia"))

# Crear columna orden_poliza: asigna el mismo número a pólizas con igual fecha
df_final = df_final.withColumn("orden_poliza", F.dense_rank().over(window_spec))

df_final.orderBy("codCliente", "orden_poliza").show(100, truncate=False)



+----------+---------+-----------------+------------+---------------------+-------------------+----------------+-------+---------------+--------+------------------------+--------+-------------------+----+------------+------------+--------------------------------------------------------+-------------------+-------------+--------------+------------+
|codCliente|codPoliza|formaPagoVigencia|valorTotal  |Valor_Total_Periodico|fechaInicioVigencia|fechaFinVigencia|estado |OpracionesGrupo|Vigencia|EstadoRenovacionesinicio|ciudad  |tipoVinculacion    |Edad|Tipo_Empresa|tomador     |nomProducto                                             |nomGrupoEmpresarial|Ramo         |orden_vigencia|orden_poliza|
+----------+---------+-----------------+------------+---------------------+-------------------+----------------+-------+---------------+--------+------------------------+--------+-------------------+----+------------+------------+--------------------------------------------------------+-------------

In [38]:
# Ordenar y numerar las pólizas por cliente y ramo según la fecha más antigua
w_ramo = Window.partitionBy("codCliente", "Ramo").orderBy(F.col("fechaInicioVigencia").asc())

df_final = df_final.withColumn(
    "rn_ramo",
    F.row_number().over(w_ramo)
)

# Filtrar para quedarnos solo con la primera póliza (más antigua) por ramo y cliente
df_final = df_final.filter(F.col("rn_ramo") == 1).drop("rn_ramo")

# Volver a recalcular orden_poliza ahora con las pólizas restantes
w_poliza = Window.partitionBy("codCliente").orderBy(F.col("fechaInicioVigencia").asc())

df_final = df_final.withColumn(
    "orden_poliza",
    F.dense_rank().over(w_poliza)
)

# Resultado ordenado para revisar visualmente
df_final.orderBy("codCliente", "orden_poliza").show(truncate=False)


+----------+---------+-----------------+-----------+---------------------+-------------------+----------------+-------+---------------+--------+------------------------+--------+-------------------+----+------------+------------+--------------------------------------------------------+-------------------+-------------+--------------+------------+
|codCliente|codPoliza|formaPagoVigencia|valorTotal |Valor_Total_Periodico|fechaInicioVigencia|fechaFinVigencia|estado |OpracionesGrupo|Vigencia|EstadoRenovacionesinicio|ciudad  |tipoVinculacion    |Edad|Tipo_Empresa|tomador     |nomProducto                                             |nomGrupoEmpresarial|Ramo         |orden_vigencia|orden_poliza|
+----------+---------+-----------------+-----------+---------------------+-------------------+----------------+-------+---------------+--------+------------------------+--------+-------------------+----+------------+------------+--------------------------------------------------------+----------------

In [39]:

# Ventana por cliente, ordenada por la secuencia de las pólizas
w = Window.partitionBy("codCliente").orderBy("orden_poliza")

# Crear lista de ramos anteriores a cada fila
df_final = df_final.withColumn(
    "ramos_previos",
    F.collect_list("Ramo").over(w.rowsBetween(Window.unboundedPreceding, -1))
)

# Reemplazar null por lista vacía
df_final = df_final.withColumn(
    "ramos_previos",
    F.when(F.col("ramos_previos").isNull(), F.array()).otherwise(F.col("ramos_previos"))
)

# Crear columna objetivo Y (siguiente ramo que compró el cliente)
df_final = df_final.withColumn(
    "Y",
    F.lead("Ramo").over(w)
)

# Obtener lista de ramos únicos y sus nombres limpios
ramos_unicos = [r["Ramo"] for r in df_final.select("Ramo").distinct().collect() if r["Ramo"] is not None]
ramos_unicos_limpios = [limpiar_nombre(r) for r in ramos_unicos]

# Crear columnas binarias (1 si ya lo compró antes O si es el actual)
for ramo_original, ramo_col in zip(ramos_unicos, ramos_unicos_limpios):
    df_final = df_final.withColumn(
        ramo_col,
        F.when(
            (F.array_contains(F.col("ramos_previos"), F.lit(ramo_original))) |
            (F.col("Ramo") == ramo_original),
            1
        ).otherwise(0)
    )

# Eliminar columna auxiliar
df_final = df_final.drop("ramos_previos")

df_final.orderBy("codCliente", "orden_poliza").select(
    "codCliente", "orden_poliza", *ramos_unicos_limpios, "Y"
).show(50, truncate=False)


+----------+------------+-----+----+-----+------------+-------------+-----+-------------+
|codCliente|orden_poliza|Salud|Vida|Autos|Cumplimiento|Patrimoniales|Otros|Y            |
+----------+------------+-----+----+-----+------------+-------------+-----+-------------+
|37        |1           |0    |1   |0    |0           |0            |0    |Autos        |
|37        |2           |0    |1   |1    |0           |0            |0    |Salud        |
|37        |3           |1    |1   |1    |0           |0            |0    |NULL         |
|39        |1           |1    |0   |0    |0           |0            |0    |NULL         |
|58        |1           |0    |0   |1    |0           |0            |0    |NULL         |
|61        |1           |0    |0   |1    |0           |0            |0    |NULL         |
|84        |1           |0    |0   |1    |0           |0            |0    |Vida         |
|84        |2           |0    |1   |1    |0           |0            |0    |NULL         |
|85       

In [40]:
df_final = df_final.filter(F.col("Y").isNotNull())
print("Número de registros:", df_final.count())

df_cliente = df_final.filter(F.col("codCliente") == "1500")
df_cliente.show(truncate=False)

Número de registros: 9316
+----------+---------+-----------------+----------+---------------------+-------------------+----------------+-------+---------------+--------+------------------------+--------+-------------------+----+------------+------------+-----------+-------------------+----+--------------+------------+-----+-----+----+-----+------------+-------------+-----+
|codCliente|codPoliza|formaPagoVigencia|valorTotal|Valor_Total_Periodico|fechaInicioVigencia|fechaFinVigencia|estado |OpracionesGrupo|Vigencia|EstadoRenovacionesinicio|ciudad  |tipoVinculacion    |Edad|Tipo_Empresa|tomador     |nomProducto|nomGrupoEmpresarial|Ramo|orden_vigencia|orden_poliza|Y    |Salud|Vida|Autos|Cumplimiento|Patrimoniales|Otros|
+----------+---------+-----------------+----------+---------------------+-------------------+----------------+-------+---------------+--------+------------------------+--------+-------------------+----+------------+------------+-----------+-------------------+----+---------

In [41]:
#Conservar solo las columnas necesarias
df_final = df_final.select(
    "codCliente",
    "codPoliza",
    "Ramo",
    "Y",
    "Salud",
    "Vida",
    "Autos",
    "Cumplimiento",
    "Patrimoniales",
    "Otros"
)
df_final.show()

+----------+---------+-------------+-------------+-----+----+-----+------------+-------------+-----+
|codCliente|codPoliza|         Ramo|            Y|Salud|Vida|Autos|Cumplimiento|Patrimoniales|Otros|
+----------+---------+-------------+-------------+-----+----+-----+------------+-------------+-----+
|        37|       69|         Vida|        Autos|    0|   1|    0|           0|            0|    0|
|        37|    80568|        Autos|        Salud|    0|   1|    1|           0|            0|    0|
|        84|   118023|        Autos|         Vida|    0|   0|    1|           0|            0|    0|
|        87|       54|         Vida|        Salud|    0|   1|    0|           0|            0|    0|
|      1356|      206| Cumplimiento|Patrimoniales|    0|   0|    0|           1|            0|    0|
|      1356|     4695|Patrimoniales|        Salud|    0|   0|    0|           1|            1|    0|
|      1366|   118014|Patrimoniales| Cumplimiento|    0|   0|    0|           0|           

In [42]:

#Trusted
# Ruta donde guardaremos la tabla unida
ruta_salida = "/content/drive/MyDrive/Dataset PI/Trust/trust_temp.csv"

# Guardar el dataframe como CSV con encabezado
df_final.coalesce(1) \
    .write.mode("overwrite") \
    .option("header", "true") \
    .csv(ruta_salida)

print(" Tabla final guardada en:", ruta_salida)



# Buscar el archivo generado dentro de la carpeta
archivo_generado = glob.glob(ruta_salida + "/part-*.csv")[0]

# Moverlo a la misma carpeta con el nombre tabla_final.csv
ruta_final = "/content/drive/MyDrive/Dataset PI/Trust/trust_data.csv"
shutil.move(archivo_generado, ruta_final)

# Eliminar la carpeta vacía
shutil.rmtree(ruta_salida)


 Tabla final guardada en: /content/drive/MyDrive/Dataset PI/Trust/trust_temp.csv
